In [ ]:
import pandas as pd
import os

testcase_dir = r"D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审"

testcase_paths = [
    os.path.join(testcase_dir, file)
    for file in os.listdir(testcase_dir)
    if file.endswith(".xlsx") and file.startswith("C")
]
testcase_paths.sort(reverse=False)


ref_target_df = pd.DataFrame(
    {
        "原始菜名": [],
        "建议菜名": [],
        "是否删除": [],
    }
)

for file in testcase_paths:
    print(f"读取文件：{file}")
    filename = os.path.basename(file).rstrip(".xlsx")
    # if not (
    #     (
    #         filename.startswith("C5")
    #         # and filename.endswith("merged.xlsx")
    #     )
    #     or filename.startswith("C3")
    # ):
    #     continue
    df = pd.read_excel(file, sheet_name=0)
    # print(df)
    temp_df = pd.DataFrame(
        {
            "原始菜名": [],
            "建议菜名": [],
            "是否删除": [],
            "测试集名称": [],
        }
    )
    # print(df.columns)
    temp_df["原始菜名"] = df["菜品名"]
    temp_df["建议菜名"] = df["建议菜名"]
    temp_df["是否删除"] = df["是否删除"]
    temp_df["测试集名称"] = [filename for _ in range(len(df))]
    add_list = df[df["替换菜名"].notnull()]["替换菜名"].tolist()
    # print(add_list)
    add_df = pd.DataFrame(
        {
            "原始菜名": add_list,
            "建议菜名": add_list,
            "是否删除": [False for _ in add_list],
            "测试集名称": [filename for _ in add_list],
        }
    )
    temp_df = pd.concat([temp_df, add_df])
    # print(temp_df.tail(10))
    temp_df.reset_index(drop=True, inplace=True)
    for i in range(len(temp_df)):
        if pd.isna(temp_df.iloc[i]["建议菜名"]):
            temp_df.loc[i, "建议菜名"] = temp_df.iloc[i]["原始菜名"]
        if pd.isna(temp_df.iloc[i]["原始菜名"]) and not pd.isna(
            temp_df.iloc[i]["建议菜名"]
        ):
            temp_df.loc[i, "原始菜名"] = temp_df.iloc[i]["建议菜名"]
        if pd.isna(temp_df.iloc[i]["是否删除"]) or temp_df.iloc[i]["是否删除"] == 0:
            temp_df.loc[i, "是否删除"] = False
        else:
            temp_df.loc[i, "是否删除"] = True
    # print(temp_df.tail(10))

    ref_target_df = pd.concat([ref_target_df, temp_df])
    # print(temp_df)
ref_path = os.path.join(testcase_dir, "ref.csv")
ref_target_df.to_csv(
    ref_path,
    index=False,
    encoding="gbk",
)
print("保存文件：ref.csv")

读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C2_testcase_lunch_and_dinner_freq5_familarity5.xlsx


c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\crwu\AppData\Local\Temp\ipykernel_52024\2239189801.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp_df.loc[i, "是否删除"] = False
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\crwu\AppData\Local\Temp\ipykernel_52024\2239189801.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  t

读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C3_testcase_memory_freq3_familarity4.xlsx
读取文件：D:\crwu\data\foodname_classification\测试集V2.1\已标注\已评审\C5_testcase_main_food_freq4_familarity5.xlsx
保存文件：ref.csv


C:\Users\crwu\AppData\Local\Temp\ipykernel_52024\2239189801.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  temp_df.loc[i, "是否删除"] = True


In [4]:
ref_target_df_2 = ref_target_df[ref_target_df["是否删除"] == False]
ref_target_df_2.insert(2, "是否汇聚", [True for _  in range(len(ref_target_df_2))])
for index, row in ref_target_df_2.iterrows():
    if row["原始菜名"] == row["建议菜名"]:
        ref_target_df_2.loc[index, "是否汇聚"] = False
ref_path = os.path.join(testcase_dir, "ref2.csv")
ref_target_df_2.to_csv(
    ref_path,
    index=False,
    encoding="gbk",
)

In [ ]:
testcase_names = set(ref_target_df["测试集名称"].to_list())
for testcase_name in testcase_names:
    print(testcase_name)
    ref_target_df = pd.DataFrame(
        {
            "爬取菜名": [],
            "ground_truth": [], 
        }
    )